In [1]:
import pandas as pd
# Extract hyperlinks from descriptions
# Add them to another column
# Look at descriptions and see if people identify as Dem or Rep
# Male and Female
# Extract common jobs and such

In [2]:
df = pd.read_json('reduced_twitter_dataset.json')

In [3]:
userdb = pd.DataFrame()

In [4]:
userdb['id'] =    list(map(lambda tweet_user: tweet_user['id'], df['user']))
userdb['name'] =    list(map(lambda tweet_user: tweet_user['name'], df['user']))
userdb['location'] =    list(map(lambda tweet_user: tweet_user['location'], df['user']))
userdb['description'] =    list(map(lambda tweet_user: tweet_user['description'], df['user']))
userdb['verified'] =    list(map(lambda tweet_user: tweet_user['verified'], df['user']))
userdb['created_at'] =    list(map(lambda tweet_user: tweet_user['created_at'], df['user']))
userdb['lang'] =    list(map(lambda tweet_user: tweet_user['lang'], df['user']))

In [5]:
import re

# A function that extracts the hyperlinks from the tweet's content.
def extract_link(text):
    regex = r'https?://[^\s<>"]+|www\.[^\s<>"]+'
    match = re.search(regex, text)
    if match:
        return match.group()
    return ''

def remove_link(text):
    regex = r'https?://[^\s<>"]+|www\.[^\s<>"]+'
    new_text = re.sub(regex,'',text)
    return new_text

def replace_word(text,word):
    new_text = text
    for loop in word:
        new_text = re.sub(loop,' ',new_text)
    return new_text

# A function that checks whether a word is included in the tweet's content
def word_in_text(word, text):
    match = re.search(word, text)
    if match:
        return True
    return False

def lowercase (doc):
    doc=doc.lower()
    return doc

In [6]:
# Make this simple!!!
temp = []
temp_1 = []
for loop in range(len(userdb['description'])):
    if isinstance(userdb['description'][loop],str):
        temp.append(extract_link(userdb['description'][loop]))
        temp_1.append(remove_link(userdb['description'][loop]))
    else:
        temp.append('')
        temp_1.append('')
        


userdb['link'] = temp        
userdb['description'] = temp_1  




In [7]:
temp_2 = []
for loop in range(len(userdb['description'])):
    if isinstance(userdb['description'][loop],str):
        temp_2.append(   lowercase(userdb['description'][loop])     )
    else:
        temp_2.append('')        


In [8]:
userdb['description'] = temp_2

In [9]:
userdb_reduced = userdb.drop_duplicates()
len(userdb_reduced)

93282

In [10]:
userdb_train = userdb_reduced
userdb_tweek = userdb_reduced[74638:83968]
userdb_test = userdb_reduced[83968:]

In [11]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string


stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
exclude.remove('#')
stop.update(['im','like','de','la','en','que','el','e','mi','del','un',"i'm",'los','sc','u','le','here','you','19','20','la','por','one','new','alum','tweet','own','former'])

def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

lemma = WordNetLemmatizer()

In [13]:
clean_1 = []

for loop in range(len(userdb_train)):
    clean_1.append(clean(userdb_train.iloc[loop]['description']))    

userdb_train.loc[:,'description'] = clean_1

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [14]:
doc_clean = [clean(doc).split() for doc in userdb_train[userdb_train['lang'] == 'en']['description']]

In [15]:
# Importing Gensim
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index
dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [16]:
print (doc_clean[1])
print (dictionary[16])
print (doc_term_matrix[1])

['awscwi', 'pipefitter', 'usaf', 'nra', 'remember', 'usa', 'hunting', 'fishing', 'harleydavidsons', 'time', 'h', 'student', 'could', 'carry', 'gun', 'trks', '2', 'school']
could
[(13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1)]


In [23]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=8, id2word = dictionary, passes=10)

In [20]:
# Print 1: Looked at all the english tweets and split it up into 4 topics??
topics = ldamodel.print_topics(num_topics=8, num_words=8)
# coherence score 
i=0
for topic in topics:
    print ("Topic",i ,"->", topic)     
    i+=1


Topic 0 -> (0, '0.045*"year" + 0.028*"old" + 0.021*"hard" + 0.018*"fan" + 0.016*"yr" + 0.015*"working" + 0.013*"w" + 0.010*"die"')
Topic 1 -> (1, '0.034*"thing" + 0.032*"right" + 0.028*"human" + 0.026*"law" + 0.024*"better" + 0.014*"making" + 0.013*"place" + 0.012*"around"')
Topic 2 -> (2, '0.079*"fan" + 0.057*"lover" + 0.052*"•" + 0.034*"enthusiast" + 0.024*"food" + 0.021*"animal" + 0.019*"sport" + 0.016*"geek"')
Topic 3 -> (3, '0.019*"senior" + 0.013*"phd" + 0.012*"going" + 0.010*"relation" + 0.009*"snap" + 0.009*"believer" + 0.009*"assistant" + 0.009*"earth"')
Topic 4 -> (4, '0.033*"since" + 0.032*"endorsement" + 0.022*"stuff" + 0.021*"rts" + 0.019*"executive" + 0.019*"state" + 0.013*"≠" + 0.013*"n"')
Topic 5 -> (5, '0.079*"love" + 0.047*"music" + 0.025*"art" + 0.022*"jesus" + 0.021*"book" + 0.020*"travel" + 0.017*"nerd" + 0.016*"game"')
Topic 6 -> (6, '0.031*"view" + 0.029*"opinion" + 0.029*"university" + 0.017*"student" + 0.015*"public" + 0.014*"education" + 0.014*"political" + 0.

In [144]:
def extract(words):
    if words != '+':
        num_1 = words.find('*')
        num = int((float(words[0:num_1]) * 1000)/2)
        num_1 = words.find('"')
        word = words[num_1+1:-1]
        return num, word
    return 0,''

In [214]:
res = [[],[],[],[],[],[],[],[]]
for loop in range(8):
    res[loop] = topics[loop][1].split() 

In [216]:
res = [[],[],[],[],[],[],[],[]]
res[0] =  '0.022*today" + 0.019*"great" + 0.013*"pressure" + 0.013*"rain" + 0.013*"forecast" + 0.013*"orchard" + 0.013*"tempcrab" + 0.012*"0in" + 0.011*"fine" + 0.010*"american"'
res[1] = '0.022*"tax" + 0.015*"amp" + 0.010*"god" + 0.008*"release" + 0.007*"time" + 0.007*"return" + 0.007*"idiot" + 0.006*"money" + 0.006*"them" + 0.005*"email"'
res[2] ='0.010*"plan" + 0.010*"question" + 0.008*"hate" + 0.008*"trump" + 0.008*"press" + 0.007*"me" + 0.006*"sad" + 0.006*"yes" + 0.006*"fuck" + 0.006*"answer"'
res[3] = '0.028*"" + 0.008*"president" + 0.006*"open" + 0.006*"evil" + 0.005*"problem" + 0.005*"honest" + 0.004*"border" + 0.004*"happy" + 0.004*"killed" + 0.004*"ha"'
res[4] = '0.019*"it" + 0.013*"true" + 0.010*"talk" + 0.007*"usa" + 0.007*"wow" + 0.007*"russia" + 0.006*"friend" + 0.006*"war" + 0.005*"public" + 0.004*"idea"'
res[5] = '0.027*"hillary" + 0.023*"amp" + 0.018*"clinton" + 0.018*"vote" + 0.011*"good" + 0.008*"him" + 0.008*"obama" + 0.007*"remember" + 0.006*"elected" + 0.006*"foundation"'
res[6] ='0.050*"trump" + 0.018*"amp" + 0.015*"people" + 0.011*"vote" + 0.011*"donald" + 0.010*"hillary" + 0.010*"love" + 0.010*"campaign" + 0.010*"america" + 0.009*"racist"'
res[7] ='0.016*"trump" +0.012*"wall" + 0.011*"pay" + 0.010*"liar" + 0.010*"work" + 0.009*"amp" + 0.008*"big" + 0.007*"day" + 0.006*"it" + 0.006*"mexico"'

In [217]:
for loop in range(8):
    res[loop] = res[loop].split() 

list

In [218]:
topic_string = ['','','','','','','','']
i = 0
for topics_string in (res):
    for words_words in (topics_string):
        num , word = extract(words_words)
        temp = ''
        for loop in range(num):
               temp = temp + ' ' + word
        topic_string[i] =  topic_string[i] + ' ' + temp
    
    i = i + 1

In [219]:
topic_string[0]

'               great great great great great great great great great   pressure pressure pressure pressure pressure pressure   rain rain rain rain rain rain   forecast forecast forecast forecast forecast forecast   orchard orchard orchard orchard orchard orchard   tempcrab tempcrab tempcrab tempcrab tempcrab tempcrab   0in 0in 0in 0in 0in 0in   fine fine fine fine fine   american american american american american'

In [22]:


from wordcloud import WordCloud, STOPWORDS

wordcloud = WordCloud(width=800, height=400).generate(topic_string[0])
plt.figure( figsize=(20,10) )
plt.imshow(wordcloud)

NameError: name 'topic_string' is not defined